In [92]:
"""
Code modified from PyTorch DCGAN examples: https://github.com/pytorch/examples/tree/master/dcgan
"""
from __future__ import print_function
import argparse
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [93]:

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# compute the current classification accuracy
def compute_acc(preds, labels):
    correct = 0
    preds_ = preds.data.max(1)[1]
    correct = preds_.eq(labels.data).cpu().sum()
    acc = float(correct) / float(len(labels.data)) * 100.0
    return acc

In [94]:
import torch.utils.data as data

from PIL import Image
import os
import os.path

IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm']


def is_image_file(filename):
    """Checks if a file is an image.

    Args:
        filename (string): path to a file

    Returns:
        bool: True if the filename ends with a known image extension
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in IMG_EXTENSIONS)


def find_classes(dir, classes_idx=None):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    if classes_idx is not None:
        assert type(classes_idx) == tuple
        start, end = classes_idx
        classes = classes[start:end]
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


def make_dataset(dir, class_to_idx):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(os.listdir(dir)):
        if target not in class_to_idx:
            continue
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


class ImageFolder(data.Dataset):
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png

    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.

     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader, classes_idx=None):
        self.classes_idx = classes_idx
        classes, class_to_idx = find_classes(root, self.classes_idx)
        imgs = make_dataset(root, class_to_idx)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(IMG_EXTENSIONS)))

        self.root = root
        self.imgs = imgs
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.imgs)


In [95]:
class _netG(nn.Module):
    def __init__(self, ngpu, nz):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        self.nz = nz

        # first linear layer
        self.fc1 = nn.Linear(110, 768)
        # Transposed Convolution 2
        self.tconv2 = nn.Sequential(
            nn.ConvTranspose2d(768, 384, 5, 2, 0, bias=False),
            nn.BatchNorm2d(384),
            nn.ReLU(True),
        )
        # Transposed Convolution 3
        self.tconv3 = nn.Sequential(
            nn.ConvTranspose2d(384, 256, 5, 2, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )
        # Transposed Convolution 4
        self.tconv4 = nn.Sequential(
            nn.ConvTranspose2d(256, 192, 5, 2, 0, bias=False),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )
        # Transposed Convolution 5
        self.tconv5 = nn.Sequential(
            nn.ConvTranspose2d(192, 64, 5, 2, 0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
        )
        # Transposed Convolution 5
        self.tconv6 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, 8, 2, 0, bias=False),
            nn.Tanh(),
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            input = input.view(-1, self.nz)
            fc1 = nn.parallel.data_parallel(self.fc1, input, range(self.ngpu))
            fc1 = fc1.view(-1, 768, 1, 1)
            tconv2 = nn.parallel.data_parallel(self.tconv2, fc1, range(self.ngpu))
            tconv3 = nn.parallel.data_parallel(self.tconv3, tconv2, range(self.ngpu))
            tconv4 = nn.parallel.data_parallel(self.tconv4, tconv3, range(self.ngpu))
            tconv5 = nn.parallel.data_parallel(self.tconv5, tconv4, range(self.ngpu))
            tconv5 = nn.parallel.data_parallel(self.tconv6, tconv5, range(self.ngpu))
            output = tconv5
        else:
            input = input.view(-1, self.nz)
            fc1 = self.fc1(input)
            fc1 = fc1.view(-1, 768, 1, 1)
            tconv2 = self.tconv2(fc1)
            tconv3 = self.tconv3(tconv2)
            tconv4 = self.tconv4(tconv3)
            tconv5 = self.tconv5(tconv4)
            tconv5 = self.tconv6(tconv5)
            output = tconv5
        return output


class _netD(nn.Module):
    def __init__(self, ngpu, num_classes=10):
        super(_netD, self).__init__()
        self.ngpu = ngpu

        # Convolution 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 0, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 3
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 4
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 0, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 5
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 6
        self.conv6 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # discriminator fc
        self.fc_dis = nn.Linear(13*13*512, 1)
        # aux-classifier fc
        self.fc_aux = nn.Linear(13*13*512, num_classes)
        # softmax and sigmoid
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            conv1 = nn.parallel.data_parallel(self.conv1, input, range(self.ngpu))
            conv2 = nn.parallel.data_parallel(self.conv2, conv1, range(self.ngpu))
            conv3 = nn.parallel.data_parallel(self.conv3, conv2, range(self.ngpu))
            conv4 = nn.parallel.data_parallel(self.conv4, conv3, range(self.ngpu))
            conv5 = nn.parallel.data_parallel(self.conv5, conv4, range(self.ngpu))
            conv6 = nn.parallel.data_parallel(self.conv6, conv5, range(self.ngpu))
            flat6 = conv6.view(-1, 13*13*512)
            fc_dis = nn.parallel.data_parallel(self.fc_dis, flat6, range(self.ngpu))
            fc_aux = nn.parallel.data_parallel(self.fc_aux, flat6, range(self.ngpu))
        else:
            conv1 = self.conv1(input)
            conv2 = self.conv2(conv1)
            conv3 = self.conv3(conv2)
            conv4 = self.conv4(conv3)
            conv5 = self.conv5(conv4)
            conv6 = self.conv6(conv5)
            flat6 = conv6.view(-1, 13*13*512)
            fc_dis = self.fc_dis(flat6)
            fc_aux = self.fc_aux(flat6)
        classes = self.softmax(fc_aux)
        realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)
        return realfake, classes


class _netG_CIFAR10(nn.Module):
    def __init__(self, ngpu, nz):
        super(_netG_CIFAR10, self).__init__()
        self.ngpu = ngpu
        self.nz = nz

        # first linear layer
        self.fc1 = nn.Linear(110, 384)
        # Transposed Convolution 2
        self.tconv2 = nn.Sequential(
            nn.ConvTranspose2d(384, 192, 4, 1, 0, bias=False),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )
        # Transposed Convolution 3
        self.tconv3 = nn.Sequential(
            nn.ConvTranspose2d(192, 96, 4, 2, 1, bias=False),
            nn.BatchNorm2d(96),
            nn.ReLU(True),
        )
        # Transposed Convolution 4
        self.tconv4 = nn.Sequential(
            nn.ConvTranspose2d(96, 48, 4, 2, 1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(True),
        )
        # Transposed Convolution 4
        self.tconv5 = nn.Sequential(
            nn.ConvTranspose2d(48, 3, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            input = input.view(-1, self.nz)
            fc1 = nn.parallel.data_parallel(self.fc1, input, range(self.ngpu))
            fc1 = fc1.view(-1, 384, 1, 1)
            tconv2 = nn.parallel.data_parallel(self.tconv2, fc1, range(self.ngpu))
            tconv3 = nn.parallel.data_parallel(self.tconv3, tconv2, range(self.ngpu))
            tconv4 = nn.parallel.data_parallel(self.tconv4, tconv3, range(self.ngpu))
            tconv5 = nn.parallel.data_parallel(self.tconv5, tconv4, range(self.ngpu))
            output = tconv5
        else:
            input = input.view(-1, self.nz)
            fc1 = self.fc1(input)
            fc1 = fc1.view(-1, 384, 1, 1)
            tconv2 = self.tconv2(fc1)
            tconv3 = self.tconv3(tconv2)
            tconv4 = self.tconv4(tconv3)
            tconv5 = self.tconv5(tconv4)
            output = tconv5
        return output


class _netD_CIFAR10(nn.Module):
    def __init__(self, ngpu, num_classes=10):
        super(_netD_CIFAR10, self).__init__()
        self.ngpu = ngpu

        # Convolution 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 3
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 4
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 5
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # Convolution 6
        self.conv6 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5, inplace=False),
        )
        # discriminator fc
        self.fc_dis = nn.Linear(4*4*512, 1)
        # aux-classifier fc
        self.fc_aux = nn.Linear(4*4*512, num_classes)
        # softmax and sigmoid
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            conv1 = nn.parallel.data_parallel(self.conv1, input, range(self.ngpu))
            conv2 = nn.parallel.data_parallel(self.conv2, conv1, range(self.ngpu))
            conv3 = nn.parallel.data_parallel(self.conv3, conv2, range(self.ngpu))
            conv4 = nn.parallel.data_parallel(self.conv4, conv3, range(self.ngpu))
            conv5 = nn.parallel.data_parallel(self.conv5, conv4, range(self.ngpu))
            conv6 = nn.parallel.data_parallel(self.conv6, conv5, range(self.ngpu))
            flat6 = conv6.view(-1, 4*4*512)
            fc_dis = nn.parallel.data_parallel(self.fc_dis, flat6, range(self.ngpu))
            fc_aux = nn.parallel.data_parallel(self.fc_aux, flat6, range(self.ngpu))
        else:
            conv1 = self.conv1(input)
            conv2 = self.conv2(conv1)
            conv3 = self.conv3(conv2)
            conv4 = self.conv4(conv3)
            conv5 = self.conv5(conv4)
            conv6 = self.conv6(conv5)
            flat6 = conv6.view(-1, 4*4*512)
            fc_dis = self.fc_dis(flat6)
            fc_aux = self.fc_aux(flat6)
        classes = self.softmax(fc_aux)
        realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)
        return realfake, classes


In [96]:
dataset = 'cifar10'
dataroot = './root'
workers = 32
batchSize = 64
imageSize = 32
nz = 110
ngf = 64
ndf = 64
niter = 25
lr = 0.0002
beta1 = 0.5
cuda = False
ngpu = 1
netG = ''
netD = ''
outf = '.'
manualSeed = 1
num_classes = 10
gpu_id = 0


In [97]:
# specify the gpu id if using only 1 gpu
if ngpu == 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)

In [98]:
'''if manualSeed is None:
    manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if cuda:
    torch.cuda.manual_seed_all(manualSeed)'''

'if manualSeed is None:\n    manualSeed = random.randint(1, 10000)\nprint("Random Seed: ", manualSeed)\nrandom.seed(manualSeed)\ntorch.manual_seed(manualSeed)\nif cuda:\n    torch.cuda.manual_seed_all(manualSeed)'

In [99]:
dataset = dset.CIFAR10(
        root=dataroot, download=True,
        transform=transforms.Compose([
            transforms.Scale(imageSize),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]))

/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


Files already downloaded and verified


In [100]:
assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, num_workers=int(workers))


In [101]:
# some hyper parameters
ngpu = int(ngpu)
nz = int(nz)
ngf = int(ngf)
ndf = int(ndf)
num_classes = int(num_classes)
nc = 3

In [102]:
netG = _netG_CIFAR10(ngpu, nz)
netG.apply(weights_init)

netD = _netD_CIFAR10(ngpu, num_classes)
netD.apply(weights_init)

_netD_CIFAR10(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Dropout(p=0.5)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Dropout(p=0.5)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Dropout(p=0.5)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
 

In [103]:
# loss functions
dis_criterion = nn.BCELoss()
aux_criterion = nn.NLLLoss()

In [104]:
# tensor placeholders
input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
eval_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
dis_label = torch.FloatTensor(batchSize)
aux_label = torch.LongTensor(batchSize)
real_label = 1
fake_label = 0

In [105]:
# if using cuda
if cuda:
    netD.cuda()
    netG.cuda()
    dis_criterion.cuda()
    aux_criterion.cuda()
    input, dis_label, aux_label = input.cuda(), dis_label.cuda(), aux_label.cuda()
    noise, eval_noise = noise.cuda(), eval_noise.cuda()

In [106]:
# define variables
input = Variable(input)
noise = Variable(noise)
eval_noise = Variable(eval_noise)
dis_label = Variable(dis_label)
aux_label = Variable(aux_label)
# noise for evaluation
eval_noise_ = np.random.normal(0, 1, (batchSize, nz))
eval_label = np.random.randint(0, num_classes, batchSize)
eval_onehot = np.zeros((batchSize, num_classes))
eval_onehot[np.arange(batchSize), eval_label] = 1
eval_noise_[np.arange(batchSize), :num_classes] = eval_onehot[np.arange(batchSize)]
eval_noise_ = (torch.from_numpy(eval_noise_))
eval_noise.data.copy_(eval_noise_.view(batchSize, nz, 1, 1))

tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[ 1.0000]],

         ...,

         [[-0.6634]],

         [[-1.6777]],

         [[-0.7255]]],


        [[[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         ...,

         [[ 1.9469]],

         [[-1.3862]],

         [[-0.0978]]],


        [[[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         ...,

         [[ 0.2930]],

         [[-1.0074]],

         [[ 0.3345]]],


        ...,


        [[[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         ...,

         [[-0.0825]],

         [[-0.8512]],

         [[ 1.4252]]],


        [[[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         ...,

         [[-1.5785]],

         [[ 0.3623]],

         [[ 0.8861]]],


        [[[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         ...,

         [[-0.4978]],

         [[ 0.8109]],

         [[-0.2708]]]])

In [107]:
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [108]:
avg_loss_D = 0.0
avg_loss_G = 0.0
avg_loss_A = 0.0

In [109]:
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, label = data
        batch_size = real_cpu.size(0)
        if cuda:
            real_cpu = real_cpu.cuda()
        input.data.resize_as_(real_cpu).copy_(real_cpu)
        dis_label.data.resize_(batch_size).fill_(real_label)
        aux_label.data.resize_(batch_size).copy_(label)
        dis_output, aux_output = netD(input)

        dis_errD_real = dis_criterion(dis_output, dis_label)
        aux_errD_real = aux_criterion(aux_output, aux_label)
        errD_real = dis_errD_real + aux_errD_real
        errD_real.backward()
        D_x = dis_output.data.mean()

        # compute the current classification accuracy
        accuracy = compute_acc(aux_output, aux_label)

        # train with fake
        noise.data.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        label = np.random.randint(0, num_classes, batch_size)
        noise_ = np.random.normal(0, 1, (batch_size, nz))
        class_onehot = np.zeros((batch_size, num_classes))
        class_onehot[np.arange(batch_size), label] = 1
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]
        noise_ = (torch.from_numpy(noise_))
        noise.data.copy_(noise_.view(batch_size, nz, 1, 1))
        aux_label.data.resize_(batch_size).copy_(torch.from_numpy(label))

        fake = netG(noise)
        dis_label.data.fill_(fake_label)
        dis_output, aux_output = netD(fake.detach())
        dis_errD_fake = dis_criterion(dis_output, dis_label)
        aux_errD_fake = aux_criterion(aux_output, aux_label)
        errD_fake = dis_errD_fake + aux_errD_fake
        errD_fake.backward()
        D_G_z1 = dis_output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        dis_label.data.fill_(real_label)  # fake labels are real for generator cost
        dis_output, aux_output = netD(fake)
        dis_errG = dis_criterion(dis_output, dis_label)
        aux_errG = aux_criterion(aux_output, aux_label)
        errG = dis_errG + aux_errG
        errG.backward()
        D_G_z2 = dis_output.data.mean()
        optimizerG.step()

        # compute the average loss
        curr_iter = epoch * len(dataloader) + i
        all_loss_G = avg_loss_G * curr_iter
        all_loss_D = avg_loss_D * curr_iter
        all_loss_A = avg_loss_A * curr_iter
        all_loss_G += errG.data[0]
        all_loss_D += errD.data[0]
        all_loss_A += accuracy
        avg_loss_G = all_loss_G / (curr_iter + 1)
        avg_loss_D = all_loss_D / (curr_iter + 1)
        avg_loss_A = all_loss_A / (curr_iter + 1)

        print('[%d/%d][%d/%d] Loss_D: %.4f (%.4f) Loss_G: %.4f (%.4f) D(x): %.4f D(G(z)): %.4f / %.4f Acc: %.4f (%.4f)'
              % (epoch, niter, i, len(dataloader),
                 errD.data[0], avg_loss_D, errG.data[0], avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))
        if i % 100 == 0:
            vutils.save_image(
                real_cpu, '%s/real_samples.png' % outf)
            print('Label for eval = {}'.format(eval_label))
            fake = netG(eval_noise)
            vutils.save_image(
                fake.data,
                '%s/fake_samples_epoch_%03d.png' % (outf, epoch)
            )

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outf, epoch))

/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:272: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/25][0/782] Loss_D: 1.3390 (1.3390) Loss_G: 0.7419 (0.7419) D(x): 0.4662 D(G(z)): 0.5029 / 0.4509 Acc: 10.9375 (10.9375)
Label for eval = [2 5 9 8 7 2 8 1 8 7 5 9 8 3 5 4 9 8 7 5 7 3 0 9 7 9 4 2 7 9 0 5 6 0 1 2 2
 0 0 9 4 0 0 6 1 2 1 5 3 6 9 8 1 1 7 4 3 8 1 6 7 8 6 5]
[0/25][1/782] Loss_D: 1.2310 (1.2850) Loss_G: 0.6641 (0.7030) D(x): 0.4897 D(G(z)): 0.4767 / 0.4794 Acc: 6.2500 (8.5938)
[0/25][2/782] Loss_D: 1.1786 (1.2495) Loss_G: 0.6075 (0.6712) D(x): 0.4977 D(G(z)): 0.4679 / 0.4916 Acc: 4.6875 (7.2917)
[0/25][3/782] Loss_D: 1.2894 (1.2595) Loss_G: 0.6547 (0.6671) D(x): 0.4730 D(G(z)): 0.4882 / 0.4841 Acc: 12.5000 (8.5938)
[0/25][4/782] Loss_D: 1.2981 (1.2672) Loss_G: 0.6809 (0.6698) D(x): 0.4767 D(G(z)): 0.4964 / 0.4734 Acc: 10.9375 (9.0625)
[0/25][5/782] Loss_D: 1.1291 (1.2442) Loss_G: 0.7082 (0.6762) D(x): 0.5121 D(G(z)): 0.4445 / 0.4677 Acc: 9.3750 (9.1146)
[0/25][6/782] Loss_D: 1.1675 (1.2332) Loss_G: 0.7148 (0.6817) D(x): 0.5188 D(G(z)): 0.4812 / 0.4654 Acc: 14.0625 (9.8214)


[0/25][66/782] Loss_D: 0.8870 (1.1136) Loss_G: 1.2205 (0.8563) D(x): 0.5436 D(G(z)): 0.3492 / 0.3220 Acc: 18.7500 (11.8703)
[0/25][67/782] Loss_D: 1.0121 (1.1121) Loss_G: 1.0663 (0.8594) D(x): 0.5396 D(G(z)): 0.3872 / 0.3515 Acc: 14.0625 (11.9026)
[0/25][68/782] Loss_D: 1.1275 (1.1123) Loss_G: 0.9963 (0.8613) D(x): 0.5422 D(G(z)): 0.4397 / 0.3834 Acc: 14.0625 (11.9339)
[0/25][69/782] Loss_D: 0.8887 (1.1091) Loss_G: 1.3281 (0.8680) D(x): 0.6031 D(G(z)): 0.3820 / 0.2753 Acc: 14.0625 (11.9643)
[0/25][70/782] Loss_D: 0.8533 (1.1055) Loss_G: 1.4325 (0.8760) D(x): 0.5946 D(G(z)): 0.3894 / 0.2620 Acc: 15.6250 (12.0158)
[0/25][71/782] Loss_D: 0.9417 (1.1033) Loss_G: 1.1380 (0.8796) D(x): 0.5690 D(G(z)): 0.3938 / 0.3185 Acc: 17.1875 (12.0877)
[0/25][72/782] Loss_D: 0.9341 (1.1009) Loss_G: 1.2594 (0.8848) D(x): 0.5681 D(G(z)): 0.4056 / 0.2988 Acc: 12.5000 (12.0933)
[0/25][73/782] Loss_D: 0.9043 (1.0983) Loss_G: 1.1942 (0.8890) D(x): 0.5550 D(G(z)): 0.3812 / 0.3059 Acc: 12.5000 (12.0988)
[0/25][7

[0/25][131/782] Loss_D: 1.1694 (0.9870) Loss_G: 0.8168 (1.1527) D(x): 0.6284 D(G(z)): 0.5363 / 0.4504 Acc: 14.0625 (13.6719)
[0/25][132/782] Loss_D: 1.5805 (0.9915) Loss_G: 0.6307 (1.1487) D(x): 0.5134 D(G(z)): 0.5175 / 0.5073 Acc: 12.5000 (13.6631)
[0/25][133/782] Loss_D: 1.3908 (0.9944) Loss_G: 0.6050 (1.1447) D(x): 0.5055 D(G(z)): 0.5293 / 0.5217 Acc: 23.4375 (13.7360)
[0/25][134/782] Loss_D: 1.2744 (0.9965) Loss_G: 0.6107 (1.1407) D(x): 0.5651 D(G(z)): 0.5439 / 0.4959 Acc: 17.1875 (13.7616)
[0/25][135/782] Loss_D: 1.1338 (0.9975) Loss_G: 0.8139 (1.1383) D(x): 0.6586 D(G(z)): 0.5541 / 0.4426 Acc: 21.8750 (13.8212)
[0/25][136/782] Loss_D: 1.2571 (0.9994) Loss_G: 0.6298 (1.1346) D(x): 0.5130 D(G(z)): 0.5127 / 0.4959 Acc: 21.8750 (13.8800)
[0/25][137/782] Loss_D: 1.2818 (1.0015) Loss_G: 0.7060 (1.1315) D(x): 0.5300 D(G(z)): 0.5175 / 0.4863 Acc: 20.3125 (13.9266)
[0/25][138/782] Loss_D: 1.0374 (1.0017) Loss_G: 0.7153 (1.1285) D(x): 0.5856 D(G(z)): 0.5019 / 0.4747 Acc: 15.6250 (13.9388)


[0/25][197/782] Loss_D: 1.0469 (1.0079) Loss_G: 0.8152 (1.0281) D(x): 0.5740 D(G(z)): 0.4815 / 0.4308 Acc: 15.6250 (14.8595)
[0/25][198/782] Loss_D: 1.0983 (1.0084) Loss_G: 0.6866 (1.0264) D(x): 0.5993 D(G(z)): 0.5413 / 0.4698 Acc: 25.0000 (14.9105)
[0/25][199/782] Loss_D: 0.9535 (1.0081) Loss_G: 0.7013 (1.0248) D(x): 0.6075 D(G(z)): 0.4790 / 0.4692 Acc: 21.8750 (14.9453)
[0/25][200/782] Loss_D: 1.0535 (1.0083) Loss_G: 0.8383 (1.0238) D(x): 0.6270 D(G(z)): 0.5373 / 0.4392 Acc: 20.3125 (14.9720)
Label for eval = [2 5 9 8 7 2 8 1 8 7 5 9 8 3 5 4 9 8 7 5 7 3 0 9 7 9 4 2 7 9 0 5 6 0 1 2 2
 0 0 9 4 0 0 6 1 2 1 5 3 6 9 8 1 1 7 4 3 8 1 6 7 8 6 5]
[0/25][201/782] Loss_D: 1.0870 (1.0087) Loss_G: 0.8242 (1.0228) D(x): 0.5798 D(G(z)): 0.5251 / 0.4566 Acc: 25.0000 (15.0217)
[0/25][202/782] Loss_D: 1.0303 (1.0088) Loss_G: 0.8169 (1.0218) D(x): 0.6015 D(G(z)): 0.4883 / 0.4528 Acc: 21.8750 (15.0554)
[0/25][203/782] Loss_D: 0.9627 (1.0086) Loss_G: 0.8991 (1.0212) D(x): 0.6305 D(G(z)): 0.4711 / 0.4301 

...

Process Process-555:
Process Process-572:
Process Process-570:
Process Process-561:
Process Process-557:
Process Process-562:
Process Process-566:
Process Process-567:
Process Process-571:
Process Process-556:
Process Process-552:
Process Process-576:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-549:
Process Process-575:
Process Process-569:
Process Process-548:
Process Process-546:
Process Process-551:
Process Process-563:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-574:
Process Process-565:
Traceback (most recent call last):
Process Process-568:
Process Process-545:
Process Process-553:
Process Process-573:
Process Process-550:
Process Process-558:
Process Process-554:
Process Process-547:
Process Process-560:
Traceback (most recent call last):
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
Traceback (mo

  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/rajaswa-pc/anaconda3/lib/p

  File "/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py"

  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
KeyboardInterrupt
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/rajaswa-pc/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/

KeyboardInterrupt: 